In [2]:
!pip install gputil transformers datasets torch datasets accelerate optimum


  Using cached GPUtil-1.4.0-py3-none-any.whl
  Using cached transformers-4.42.4-py3-none-any.whl.metadata (43 kB)
  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
  Using cached accelerate-0.32.1-py3-none-any.whl.metadata (18 kB)
  Using cached optimum-1.21.2-py3-none-any.whl.metadata (19 kB)
  Using cached safetensors-0.4.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached xxhash-3.4.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
Using cached transformers-4.42.4-py3-none-any.whl (9.3 MB)
Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
Using cached accelerate-0.32.1-py3-none-any.whl (314 kB)
Using cached optimum-1.21.2-py3-none-any.whl (424 kB)
Using cached requests-2.32.3-py3-none-an

In [3]:
import GPUtil

gpus = GPUtil.getGPUs()
for gpu in gpus:
    print(f"GPU ID: {gpu.id}")
    print(f"Name: {gpu.name}")
    print(f"Total Memory: {gpu.memoryTotal}MB")
    print(f"Free Memory: {gpu.memoryFree}MB")
    print(f"Used Memory: {gpu.memoryUsed}MB")
    print(f"GPU Load: {gpu.load * 100}%")


GPU ID: 0
Name: NVIDIA L40
Total Memory: 46068.0MB
Free Memory: 45373.0MB
Used Memory: 1.0MB
GPU Load: 0.0%


In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from datasets import load_dataset
import pandas as pd



In [3]:
from huggingface_hub import notebook_login

notebook_login()


In [4]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
pipe = pipeline(
    'text-generation', 
    model=model_id, 
    model_kwargs={
        "torch_dtype": torch.bfloat16,
    },
    device="cuda",
)

2024-07-14 09:28:12.834277: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-14 09:28:12.880772: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-14 09:28:13.827756: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
hellaswag_data = load_dataset("Rowan/hellaswag")

In [4]:
hellaswag_df = pd.DataFrame(hellaswag_data["train"])
hellaswag_df.drop(columns=['ind', 'ctx', 'source_id','split','split_type'], inplace = True)
hellaswag_df.head()



,activity_label,ctx_a,ctx_b,endings,label
0,Removing ice from car,"Then, the man writes over the snow covering th...",then,"[, the man adds wax to the windshield and cuts...",3
1,Baking cookies,A female chef in white uniform shows a stack o...,the pans,"[contain egg yolks and baking soda., are then ...",3
2,Baking cookies,A female chef in white uniform shows a stack o...,a knife,[is seen moving on a board and cutting out its...,3
3,Baking cookies,A tray of potatoes is loaded into the oven and...,a large tray of meat,"[is placed onto a baked potato., , ls, and pic...",3
4,Getting a haircut,The man in the center is demonstrating a hairs...,the man in the blue shirt,[is standing on the sponge cutting the hair of...,2


In [105]:
print(hellaswag_df['label'].value_counts())


label
1    10031
3    10021
0     9986
2     9867
Name: count, dtype: int64
39905


In [107]:
pd.DataFrame(hellaswag_data['test']).head()

,ind,activity_label,ctx_a,ctx_b,ctx,endings,source_id,split,split_type,label
0,14,Wakeboarding,A man is being pulled on a water ski as he flo...,he,A man is being pulled on a water ski as he flo...,[mounts the water ski and tears through the wa...,activitynet~v_-5KAycAQlC4,test,indomain,
1,71,Javelin throw,A huge crowd is in the stands in an arena. A m...,several men,A huge crowd is in the stands in an arena. A m...,"[are water boarding in a river., are shown thr...",activitynet~v_-fjUWhSM6Hc,test,indomain,
2,73,Javelin throw,The man that threw the javelin celebrates. Ano...,several men,The man that threw the javelin celebrates. Ano...,"[run out to where the javelin lands again., th...",activitynet~v_-fjUWhSM6Hc,test,indomain,
3,76,Javelin throw,The second man to throw the javelin and a man ...,the same men,The second man to throw the javelin and a man ...,[do the same action but in different locations...,activitynet~v_-fjUWhSM6Hc,test,indomain,
4,78,Javelin throw,The same men run to the the javelin's landing ...,again,The same men run to the the javelin's landing ...,"[, another man does not throw his javelin., th...",activitynet~v_-fjUWhSM6Hc,test,indomain,


In [9]:
correct = 0

# Batching can improve this
for index, row in hellaswag_df.iterrows():

    activity = row['activity_label']
    sentences = row['ctx_a']
    next_sentence_start = row['ctx_b']
    endings = row['endings']
    label = row['label']

    result = mult_choice_response(activity, sentences, next_sentence_start, endings)
    # print(result[0], label)
    
    if result[0] == label:
        correct += 1
    if index > 0 and index % 1000 == 0:
        print(f"Current accuracy: {correct/(index + 1)}");

    if index == 3:
        break


    

1


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


1
1


In [19]:
print(f"Final accuracy: {correct/len(hellaswag_df)}")


Final accuracy: 0.0


In [4]:
# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_csv("hf://datasets/Idavidrein/gpqa/gpqa_main.csv")

In [11]:
gpqa_dataset = df[['Question', 'Correct Answer', 'Incorrect Answer 1', 'Incorrect Answer 2', 'Incorrect Answer 3', 'Subdomain']]
gpqa_dataset.head()

,Question,Correct Answer,Incorrect Answer 1,Incorrect Answer 2,Incorrect Answer 3,Subdomain
0,"A large gene has dozens of exons, of which the...",R-loops,lariat,polyA tail,antisense,Molecular Biology
1,Two quantum states with energies E1 and E2 hav...,10^-4 eV,10^-11 eV,10^-8 eV\n,10^-9 eV,Physics (general)
2,trans-cinnamaldehyde was treated with methylma...,11,10,12,14,Organic Chemistry
3,how many of the following compounds exhibit op...,4,3,5,6,Organic Chemistry
4,A coating is applied to a substrate resulting ...,124°,129°,134°,139°,Chemistry (general)


In [15]:
def gpqa_answer(subject, question, options):
    
    prompt = f"You are a highly knowledgeable AI assistant capable of solving complex scienc and math problems. \
    Please read each question carefully and pick the best option.\n \
    In the response, only include the option, don't explain anything \n \
    \n\nSubject: {subject}\n\n \
    Question: {question}\n\n \
    "

    prompt += "Options:\n"
    for i, option in enumerate(options):

        option_string = f"{option}\n"
        prompt += option_string

    # print(prompt) 
    return invoke_response(prompt)

76
Accuracy0.16964285714285715


In [5]:
def invoke_batch_response(prompts):

    # Create messages for each prompt
    batch_messages = [
        [{"role": "system", "content": prompt}] for prompt in prompts
    ]
    
    terminators = [
        pipe.tokenizer.eos_token_id,
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    pipe.model.generation_config.pad_token_id = pipe.tokenizer.pad_token_id
    
    # Process the batch of messages
    outputs = pipe(
        batch_messages,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=pipe.tokenizer.eos_token_id
    )
    
    # Extract and return responses
    responses = [output[0]["generated_text"][-1]["content"] for output in outputs]
    return responses

In [50]:
def five_shot(dataset, column_name, column_value):
    # Gives 5 examples of the task given the column name and column
    # Dataset should be a pandas dataframe
    
    alphabet = [chr(i + 65) for i in range(26)]
    example_df = dataset[dataset[column_name] == column_value]

    questions = example_df['question']
    options_list = example_df['options']
    answers = example_df['answer']

    five_shot_result = "Here are some examples of the task you will be doing.\n"
    for index, (question, options, answer) in enumerate(zip(questions, options_list, answers)):
        
        five_shot_result += f"\nExample: {index + 1}\n"
        five_shot_result += f"Question: {question}\n"
        five_shot_result += "Options:\n"
        for i, option in enumerate(options):
    
            currLetter = alphabet[i]
            option_string = f"{currLetter}. {option}\n"
            five_shot_result += option_string
        five_shot_result += f"Answer: {answer}\n"
        
            
    return five_shot_result

In [57]:
def MMLU_Pro_answer(batch):

    questions = batch['question']
    options_list = batch['options']
    subjects = batch['category']

    prompts = []
    # Creating an array that maps 0 to 'A', 1 to 'B', and so on
    alphabet = [chr(i + 65) for i in range(26)]

    for subject, question, options in zip(subjects, questions, options_list):
    
        prompt = f"""You are a highly knowledgeable expert and are tasked to answer a multiple choice question\n"""
        prompt += five_shot(validation_dataset.to_pandas(), 'category', subject)
        prompt += "\n\n"
        prompt += f"Please answer the following question by providing ONLY the letter corresponding to the correct answer. Do not include the answer itself\n"
        prompt += f"Question: {question}?\n"
        prompt += "Options:\n"
        for i, option in enumerate(options):
    
            currLetter = alphabet[i]
            option_string = f"{currLetter}. {option}\n"
            prompt += option_string
            
        prompts.append(prompt)
    # print(prompts[0])
    answers = invoke_batch_response(prompts)
    return {"pred_answer": answers}

In [8]:
ds = load_dataset("TIGER-Lab/MMLU-Pro")

In [10]:
testing_dataset = ds["test"]
validation_dataset = ds['validation']



In [56]:
test_df = validation_dataset.to_pandas()

# Extract a single example and create a batch
single_example = {
    'question': [test_df.loc[0, 'question']],
    'options': [test_df.loc[0, 'options']],
    'category': [test_df.loc[0, 'category']]
}

MMLU_Pro_answer(single_example)

You are a highly knowledgeable expert and are tasked to answer a multiple choice question
Here are some examples of the task you will be doing.

Example: 1
Question: The symmetric group $S_n$ has $
actorial{n}$ elements, hence it is not true that $S_{10}$ has 10 elements.
Find the characteristic of the ring 2Z.
Options:
A. 0
B. 30
C. 3
D. 10
E. 12
F. 50
G. 2
H. 100
I. 20
J. 5
Answer: A

Example: 2
Question: Let V be the set of all real polynomials p(x). Let transformations T, S be defined on V by T:p(x) -> xp(x) and S:p(x) -> p'(x) = d/dx p(x), and interpret (ST)(p(x)) as S(T(p(x))). Which of the following is true?
Options:
A. ST + TS is the identity map of V onto itself.
B. TS = 0
C. ST = 1
D. ST - TS = 0
E. ST = T
F. ST = 0
G. ST = TS
H. ST - TS is the identity map of V onto itself.
I. TS = T
J. ST = S
Answer: H

Example: 3
Question: Let A be the set of all ordered pairs of integers (m, n) such that 7m + 12n = 22. What is the greatest negative number in the set B = {m + n : (m, n) \i

{'pred_answer': ['A']}

In [8]:
validation_dataset = validation_dataset.remove_columns(["cot_content", "src", "answer_index", "question_id"])
testing_dataset = testing_dataset.remove_columns(["cot_content", "src", "answer_index", "question_id"])

In [14]:
validation_dataset

Dataset({
    features: ['question', 'options', 'answer', 'category'],
    num_rows: 70
})

In [65]:
%%time

result_dataset = testing_dataset.map(MMLU_Pro_answer, batched=True, batch_size=64)
# result_dataset = validation_dataset.map(MMLU_Pro_answer, batched=True, batch_size=128)
result_df = result_dataset.to_pandas()


Map:   0%|          | 0/12032 [00:00<?, ? examples/s]

CPU times: user 32min 10s, sys: 2min 21s, total: 34min 32s
Wall time: 35min 23s


In [66]:
result_df.head()

,question_id,question,options,answer,answer_index,cot_content,category,src,pred_answer
0,70,"Typical advertising regulatory bodies suggest,...","[Safe practices, Fear, Jealousy, Trivial, Unsa...",I,8,,business,ori_mmlu-business_ethics,D
1,71,Managers are entrusted to run the company in t...,"[Shareholders, Diligence, Self-interest, Share...",F,5,,business,ori_mmlu-business_ethics,B
2,72,There are two main issues associated with ____...,"[Down, Autonomy, Remuneration, Benefit, Down, ...",J,9,,business,ori_mmlu-business_ethics,F
3,73,_______ locate morality beyond the sphere of r...,"[Ethical egoism, Ethics of duty, Postmodern et...",C,2,,business,ori_mmlu-business_ethics,H
4,74,Some of key differences between Islamic finan...,"[Interest, Certain, Assured, Both tangible and...",G,6,,business,ori_mmlu-business_ethics,F


In [67]:
comparison = result_df['answer'] == result_df['pred_answer']

In [68]:
comparison.value_counts()

False    8088
True     3944
Name: count, dtype: int64

In [69]:
accuracy = comparison.mean()

# Print the accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 32.78%


In [ ]:
def gpqa_answer(subject, question, options):

    subject = 
    question = 
    
    prompt = f"You are a highly knowledgeable AI assistant capable of solving complex scienc and math problems. \
    Please read each question carefully and pick the best option.\n \
    In the response, only include the option, don't explain anything \n \
    \n\nSubject: {subject}\n\n \
    Question: {question}\n\n \
    "

    prompt += "Options:\n"
    for i, option in enumerate(options):

        option_string = f"{option}\n"
        prompt += option_string

    # print(prompt) 
    return invoke_response(prompt)